In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

caffe_root = '~/caffe/' 
import sys
sys.path.insert(0, caffe_root + 'python')
import caffe

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'


DEMO_DIR = './test/AgeGenderDeepLearning'

categories = [ 'Angry' , 'Disgust' , 'Fear' , 'Happy'  , 'Neutral' ,  'Sad' , 'Surprise']

In [7]:
def showimage(im):
    if im.ndim == 3:
        im = im[:, :, ::-1]
    plt.set_cmap('jet')
    plt.imshow(im,vmin=0, vmax=0.2)
    

def vis_square(data, padsize=1, padval=0):
    data -= data.min()
    data /= data.max()
    
    # force the number of filters to be square
    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, padsize), (0, padsize)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant', constant_values=(padval, padval))
    
    # tile the filters into an image
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    
    showimage(data)

In [8]:
cur_net_dir = 'VGG_S_rgb'
# DEMO_DIR = '/home/sans/Videos/ai_reacts/test/AgeGenderDeepLearning/models'
mean_filename=os.path.join(DEMO_DIR,cur_net_dir,'mean.binaryproto')
proto_data = open(mean_filename, "rb").read()
a = caffe.io.caffe_pb2.BlobProto.FromString(proto_data)
mean  = caffe.io.blobproto_to_array(a)[0]

net_pretrained = os.path.join(DEMO_DIR,cur_net_dir,'EmotiW_VGG_S.caffemodel')
net_model_file = os.path.join(DEMO_DIR,cur_net_dir,'deploy.prototxt')
VGG_S_Net = caffe.Classifier(net_model_file, net_pretrained,
                       mean=mean,
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

In [12]:
vcap = cv2.VideoCapture(0) # 0=camera
vcopy = vcap

In [13]:
if vcopy.isOpened():
    width = vcopy.get(3)
    height = vcopy.get(4)
#     frame = frame[240:700, 250:620]
    print(width, height)

In [14]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [15]:
while True:
    if vcopy.grab():
        flag, frame = vcopy.retrieve()
#         frame = frame[240:700, 250:620] ## GoT frame part
        frame = frame[50:230, 300:550] ## Human Face - when 3 ppl are present
#         frame = frame[30:200, 120:300]
#         frame = frame[0:100, 0:200]
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(gray,
                                        scaleFactor=1.3,
                                        minNeighbors=3,
                                        minSize=(30, 30))
        if len(faces) > 0:
            for (x,y,w,h) in faces:
                cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
#                 frame = frame[y:y + h, x:x + w]

#         tight = cv2.Canny(frame, 400, 470)z
#         gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#         contours, hierarchy = cv2.findContours(tight, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#         cframe = cv2.drawContours(frame, contours, -1, (0,255,0), 3)
        
        if not flag:
            continue
        else:
            cv2.imshow('video', frame) 
            input_image = caffe.io.load_image(os.path.join(DEMO_DIR,cur_net_dir,'demo_image.png'))
            prediction = VGG_S_Net.predict([input_image],oversample=False)
            print('\rPredicted category is \t {} '.format(categories[prediction.argmax()]), end="")
    if cv2.waitKey(33) == ord('a'):
        break

KeyboardInterrupt: 